In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import sys

In [2]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [3]:
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
sys.path.append('../../../../nadavp/opt_lev_analysis/lib/')
import BeadDataFile
from discharge_tools import *

In [4]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [5]:
import bead_util as bu
# import calib_util as cal
import transfer_func_util as tf
import configuration as config

### Discharge data

In [61]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 41
fsamp = 5000
def func1_scale(i):
    bb = step_file_objs[i]
        
    freq2 = np.fft.rfftfreq(len(bb.other_data[6]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[6]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)-np.pi
    print(phi_tmp2,np.std(bb.other_data[6])*np.sqrt(2)*50)

    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [0, 100000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    bb.load_other_data()
    xx2 = signal.filtfilt(b, a, bb.pos_data[1])[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [171]:
step_cal_dir = r'/data/old_trap/20210603/bead1/discharge/fine_1/'
max_file = 500

In [183]:
step_cal_files = []
for root, dirnames, filenames in os.walk(step_cal_dir):
    for filename in fnmatch.filter(filenames, '*' + config.extensions['data']):
        if '_fpga.h5' in filename:
            continue
        step_cal_files.append(os.path.join(root, filename))
step_cal_files.sort(key = bu.find_str)
num_calib_files = len(step_cal_files)
print(num_calib_files)

# Do the step calibration
step_file_objs = []
for filname in step_cal_files[:-1]:
    try:
        df = bu.DataFile();
        df.load(filname);
        df.load_other_data()
        step_file_objs.append(df);
    except:
        print('***')

139


In [184]:
func1_scale(2)

[ 0.52248505] 6.18379312618


21473.268587354556

In [185]:
amps3 = []
for i in range(num_calib_files-1):
    print(i,'/',len(step_cal_files))
    amps3.append(func1_scale(i))

0 / 139
[ 1.46907642] 6.18310351953
1 / 139
[-1.08260669] 6.18387180454
2 / 139
[ 0.52248505] 6.18379312618
3 / 139
[-2.65506352] 6.1839834092
4 / 139
[-3.01028204] 6.184438045
5 / 139
[-0.08399147] 6.18390674379
6 / 139
[-2.60374559] 6.1842820156
7 / 139
[ 1.67527033] 6.18372182051
8 / 139
[ 2.26003184] 6.18439919339
9 / 139
[-2.70789967] 6.1845197363
10 / 139
[-2.86520288] 6.18456439037
11 / 139
[-2.98517603] 6.18521354382
12 / 139
[-2.55315195] 6.18412996861
13 / 139
[-2.36183867] 6.18432416234
14 / 139
[ 1.32091974] 6.18383867204
15 / 139
[-1.14903791] 6.18444724105
16 / 139
[ 2.06185091] 6.18401436622
17 / 139
[ 2.32184315] 6.183666994
18 / 139
[-1.22012927] 6.18453745243
19 / 139
[ 0.76361787] 6.18475149306
20 / 139
[-2.38677996] 6.18461062953
21 / 139
[ 0.49753895] 6.18423914786
22 / 139
[ 2.57170934] 6.18400722769
23 / 139
[-0.7648323] 6.18428722798
24 / 139
[ 1.08825514] 6.18368081019
25 / 139
[ 1.31710337] 6.18428127498
26 / 139
[-1.50431083] 6.18372789618
27 / 139
[ 1.592343

In [399]:
# amps = np.array(amps)
amps3 = np.array(amps3)
_,ax = plt.subplots(figsize=(9.5,4))
scale = 326.26

# Perrinf
ax.scatter(range(len(amps3)), amps3/scale, label='fit phase')
# ax.scatter(range(len(amps3)), amps3/scale, label='fit fixed-phases')

ax.set(xlabel='dataset', ylabel='Charges', ylim=(-0.3,7), xlim=(78,None))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# ax.legend()
from matplotlib.ticker import MaxNLocator
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [187]:
scale = np.abs(np.mean(amps3[87:103]))/4
print("Scale is ", scale)
bb = step_file_objs[-1]
voltage = np.std(bb.other_data[6])*np.sqrt(2)*100
print("voltage: ", voltage)

Scale is  326.524624863
voltage:  12.3684001616


In [1]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 41
def get_amp2(i):
    
    fname = r'/data/old_trap/20210603/bead1/single_electrode/wo_height_fb_25V_y_electrode_6/turbombar_powfb_xyzcool_elec6_500mV41Hz_'+str(i)+'.h5'

    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 
                  'error_A': 1, 'error_f': 10, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [0, 10000], 'limit_f': [0, 100], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.other_data[7])[::decimate]
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
#     freq2 = np.fft.rfftfreq(len(bb.other_data[6]), d=1./5000)
#     fft_angles = np.angle(np.fft.rfft(bb.other_data[6]))

    phi_tmp = m1_tmp.values[2]
#     phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
#     print(phi_tmp, phi_tmp2, m1_tmp.values[0]*50, np.std(xx2)*np.sqrt(2)*50)
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-100000, 100000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}

    xx2 = signal.filtfilt(b, a, bb.pos_data[1])[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

NameError: name 'likelihood_analyser' is not defined

In [648]:
i=0
get_amp2(i)

17459.63475178431

In [649]:
amps_elec6 = []
for i in range(20):
    print(i,'/500')
    try:
        amps_elec6.append(get_amp2(i))
    except:
        print("******")
amps_elec6 = np.array(amps_elec6)

0 /500
1 /500
2 /500
3 /500
4 /500
5 /500
6 /500
7 /500
8 /500
9 /500
10 /500
11 /500
12 /500
13 /500
14 /500
15 /500
16 /500
17 /500
18 /500
19 /500


In [651]:
response_elec5 = np.mean(amps_elec5)
response_elec6 = np.mean(amps_elec6)
print('electrodes response: ', response_elec5, response_elec6, response_elec5/response_elec6)

electrodes response:  -15190.9493979 17479.5248761 -0.869071070614


In [660]:
_,ax = plt.subplots()
ax.scatter(range(20), np.abs(amps_elec5))
ax.scatter(range(20), np.abs(amps_elec6))
x = np.mean(np.abs(amps_elec5)); dx = np.std(np.abs(amps_elec5))/np.sqrt(20)
y = np.mean(np.abs(amps_elec6)); dy = np.std(np.abs(amps_elec6))/np.sqrt(20)
z = x/y; dz = np.abs(x/y)*np.sqrt((dx/x)**2+(dy/y)**2)
print(x, dx)
print(y, dy)
print(z, dz)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

15190.9493979 45.1832023556
17479.5248761 9.8612820485
0.869071070614 0.0026310094832


### Look at neutrality data

In [1231]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 41
def get_amp(i):
    
    fname = r'/data/old_trap/20210603/bead1/neutrality2/w_height_fb_y_100V_100V_sweep_m25V_25V_spin_xz_150V_redo/turbombar_powfb_xyzcool_'+str(i)+'.h5'

    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 
                  'error_A': 1, 'error_f': 10, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [0, 10000], 'limit_f': [0, 100], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.other_data[6])[::decimate]
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
#     freq2 = np.fft.rfftfreq(len(bb.other_data[7]), d=1./5000)
#     fft_angles = np.angle(np.fft.rfft(bb.other_data[7]))

    phi_tmp = m1_tmp.values[2]
#     phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
#     print(phi_tmp, phi_tmp2, m1_tmp.values[0]*50, np.std(xx2)*np.sqrt(2)*50)
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-100000, 100000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    xx2 = signal.filtfilt(b, a, bb.pos_data[1])[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [1232]:
i=1
get_amp(i)

229.5763445120392

In [1190]:
i = 0
# get_amp(i)
fname = r'/data/old_trap/20210603/bead1/neutrality/w_height_fb_y_100V_sweep_m50v_50V_spin_xz_150V/turbombar_powfb_xyzcool_'+str(i)+'.h5'
# fname = r'/data/old_trap/20210530/bead1/neutrality/w_height_fb_100V_y_sweep_spin_xz_100kHz/turbombar_powfb_xyzcool_'+str(i)+'.h5'
bb = bu.DataFile();
bb.load(fname);
bb.load_other_data()
voltage55 = np.std(bb.other_data[6])*np.sqrt(2)*100
print("voltage: ", voltage55, np.mean(bb.other_data[6])*100, len(bb.other_data[1]))

voltage:  99.1054664302 -98.9420157168 50000


In [1236]:
amps_w_fb_spin_xz_y_100V_100V_sweep_m25V_25V_2 = []
for i in range(500):
    print(i,'/500')
    try:
        amps_w_fb_spin_xz_y_100V_100V_sweep_m25V_25V_2.append(get_amp(i))
    except:
        print("******")
amps_w_fb_spin_xz_y_100V_100V_sweep_m25V_25V_2 = np.array(amps_w_fb_spin_xz_y_100V_100V_sweep_m25V_25V_2)

0 /500
1 /500
2 /500
3 /500
4 /500
5 /500
6 /500
7 /500
8 /500
9 /500
10 /500
11 /500
12 /500
13 /500
14 /500
15 /500
16 /500
17 /500
18 /500
19 /500
20 /500
21 /500
22 /500
23 /500
24 /500
25 /500
26 /500
27 /500
28 /500
29 /500
30 /500
31 /500
32 /500
33 /500
34 /500
35 /500
36 /500
37 /500
38 /500
39 /500
40 /500
41 /500
42 /500
43 /500
44 /500
45 /500
46 /500
47 /500
48 /500
49 /500
50 /500
51 /500
52 /500
53 /500
54 /500
55 /500
56 /500
57 /500
58 /500
59 /500
60 /500
61 /500
62 /500
63 /500
64 /500
65 /500
66 /500
67 /500
68 /500
69 /500
70 /500
71 /500
72 /500
73 /500
74 /500
75 /500
76 /500
77 /500
78 /500
79 /500
80 /500
81 /500
82 /500
83 /500
84 /500
85 /500
86 /500
87 /500
88 /500
89 /500
90 /500
91 /500
92 /500
93 /500
94 /500
95 /500
96 /500
97 /500
98 /500
99 /500
100 /500
101 /500
102 /500
103 /500
104 /500
105 /500
106 /500
107 /500
108 /500
109 /500
110 /500
111 /500
112 /500
113 /500
114 /500
115 /500
116 /500
117 /500
118 /500
119 /500
120 /500
121 /500
122 /500
123

In [1038]:
voltage, voltage1, scale

(12.368400161613192, 49.500141188884037, 326.26)

In [1042]:
np.mean(amps_w_fb_spin_xz_y_elec5_50V)/np.mean(amps_w_fb_spin_xz_y_elec6_50V)

2.5261811149165512

In [825]:
print(np.mean(amps_w_fb_spin_xz_y_3_3), np.mean(amps_w_fb_spin_xz_y_2_8_3_2), np.mean(amps_w_fb_spin_xz_y_2_6_3_4))
print(np.mean(amps_w_fb_spin_xz_y_2_4_3_6), np.mean(amps_w_fb_spin_xz_y_2_2_3_8), np.mean(amps_w_fb_spin_xz_y_1_6_4))
print(np.mean(amps_w_fb_spin_xz_y_1_5_3_09), np.std(amps_w_fb_spin_xz_y_1_5_3_09)/np.sqrt(50))
print(np.mean(amps_w_fb_spin_xz_y_1_5_3_09_long), np.std(amps_w_fb_spin_xz_y_1_5_3_09_long)/np.sqrt(1000))

22.6869374827 19.6844739568 16.8302937777
12.311982932 2.93553020082 -6.97628329698
-0.140182369092 1.64115329508
-1.18209593641 0.282344573225


In [684]:
response_elec6 = np.abs(response_elec6)
response_elec5 = np.abs(response_elec5)

In [751]:
1.5*2.06

3.09

In [962]:
data = [amps_w_fb_spin_xz_y_3_3, amps_w_fb_spin_xz_y_2_8_3_2, amps_w_fb_spin_xz_y_2_6_3_4, amps_w_fb_spin_xz_y_2_4_3_6,
       amps_w_fb_spin_xz_y_2_2_3_8, amps_w_fb_spin_xz_y_1_6_4, amps_w_fb_spin_xz_y_1_5_3_09]
elec5_factor = np.array([3, 2.8, 2.6, 2.4, 2.2, 1.6, 1.5])
elec6_factor = np.array([3, 3.2, 3.4, 3.6, 3.8, 4, 3.09])
epsilon = []
_, ax= plt.subplots(1,2,figsize=(9.5,4))
for i,dd in enumerate(data):
    voltage5 = elec5_factor[i]*50/2
    voltage6 = elec6_factor[i]*50/2
    scale2 = (response_elec5+response_elec6)/(voltage5/voltage*response_elec5+voltage6/voltage*response_elec6)
    ax[0].scatter(range(50), dd/scale*scale2, label='y-'+str(voltage5)+'V-'+str(voltage6)+'V, xz-spin, w/ height fb')
    print(np.mean(dd/scale*scale2), np.std(dd/scale*scale2)/np.sqrt(50))
    epsilon.append(np.mean(dd)/scale*scale2)
ax[0].legend()
ax[1].scatter(elec6_factor/elec5_factor, epsilon)
z = np.polyfit(elec6_factor/elec5_factor, epsilon, 1)
p = np.poly1d(z)
xp = np.linspace(1, 2.5, 100)
ax[1].plot(xp, p(xp), 'r--')
ax[1].set(xlabel='Voltage ratio')
ax[0].set(ylabel=r'$\epsilon$')
print('zero at ', -z[1]/z[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.0114673827102 0.000568703356821
0.00990350223489 0.000664470952106
0.00842835130534 0.000518747317727
0.00613725511293 0.000771349745822
0.00145658950079 0.00101221291671
-0.00366800108937 0.00105923771586
-9.0429000274e-05 0.00105867701289
zero at  2.06422135515


In [1049]:
voltage5, voltage6

(25.0, 63.0)

In [1238]:
voltage5 = 3*50/2
voltage6 = 5*50/2
scale2 = (response_elec5+response_elec6)/(voltage5/voltage*response_elec5+voltage6/voltage*response_elec6)
    
data = amps_w_fb_spin_xz_y_100V_100V_sweep_m25V_25V
data2 = amps_w_fb_spin_xz_y_100V_100V_sweep_m25V_25V_2
data3 = amps_w_fb_spin_xz_y_75V_125V_sweep_m50V_50V
# data4 = amps_wo_fb_spin_xz_y_50V_0V
# data5 = amps_wo_fb_spin_xz_y_20V_79V
_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(data)), data/scale*voltage/voltage1, label='-50V to 50V bias sweep - 100V, xz-spin, w/ height fb')
ax.scatter(np.arange(len(data2)), data2/scale*voltage/voltage1, label='-50V to 50V bias sweep - 100V, xz-spin, w/ height fb')
# ax.scatter(np.arange(len(data3)), data3/scale*scale2, label='-50V to 50V bias sweep - 75V-125V, xz-spin, w/ height fb')
# ax.scatter(np.arange(len(data4))+300, data4/scale*voltage/voltage1*2, label='y-50V-0V, xz-spin, w/o height fb')
# ax.scatter(np.arange(len(data5))+400, data5/scale*voltage/voltage1*2, label='y-20V-79V, xz-spin, w/o height fb')
ax.plot(np.arange(0,500), np.arange(0,500)*0, '--r')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()
np.mean(data3)/scale*scale2, np.std(data3)/np.sqrt(len(data3))/scale*scale2

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0023465860879828518, 0.0075607924090282056)

In [827]:
_,ax = plt.subplots()
_ = ax.hist(data3/scale*scale2, bins=30)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [252]:
data = np.array(amps_wo_fb_spin_xz_y)
data2 = amps_wo_fb_spin_xz_y_50V
data3 = amps_wo_fb_spin_xz_y_25V
data4 = amps_wo_fb_spin_xz_y_15V
data5 = amps_wo_fb_spin_xz_y_10V
_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(data)), data/scale*voltage/voltage1/2, label='y-100V, xz-spin, w/o height fb')
ax.scatter(np.arange(len(data2))+100, data2/scale*voltage/voltage1, label='y-50V, xz-spin, w/o heigh fb')
ax.scatter(np.arange(len(data3))+200, data3/scale*voltage/voltage2, label='y-25V, xz-spin, w/o heigh fb')
ax.scatter(np.arange(len(data4))+300, data4/scale*voltage/voltage3, label='y-15V, xz-spin, w/o heigh fb')
ax.scatter(np.arange(len(data5))+400, data5/scale*voltage/voltage4, label='y-10V, xz-spin, w/o heigh fb')
ax.plot(np.arange(0,500), np.arange(0,500)*0, '--r')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()
# print(np.mean(data3/scale*voltage/voltage1), np.std(data3/scale*voltage/voltage1)/np.sqrt(len(data3)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Z direction

In [1151]:
i = 0
fname = r'/data/old_trap/20210603/bead1/neutrality/w_height_fb_z_100V_sweep_m50v_50V_spin_xy_150V/turbombar_powfb_xyzcool_'+str(i)+'.h5'
bb = bu.DataFile();
bb.load(fname);
bb.load_other_data()
voltage_z = np.std(bb.other_data[3])*np.sqrt(2)*100
print("voltage: ", voltage_z, np.mean(bb.other_data[2])*100, len(bb.other_data[1]))

voltage:  95.8834783873 -97.5488096437 50000


In [814]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 41
def get_amp3(i):
    
    fname = r'/data/old_trap/20210603/bead1/single_electrode/wo_height_fb_10V_z_electrode_2/turbombar_powfb_xyzcool_elec2_200mV41Hz_'+str(i)+'.h5'

    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 
                  'error_A': 1, 'error_f': 10, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [0, 10000], 'limit_f': [0, 100], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.other_data[3])[::decimate]
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
#     freq2 = np.fft.rfftfreq(len(bb.other_data[6]), d=1./5000)
#     fft_angles = np.angle(np.fft.rfft(bb.other_data[6]))

    phi_tmp = m1_tmp.values[2]
#     phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
#     print(phi_tmp, phi_tmp2, m1_tmp.values[0]*50, np.std(xx2)*np.sqrt(2)*50)
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-100000, 100000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}

    xx2 = signal.filtfilt(b, a, bb.pos_data[2])[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    print('phase response: ', m1_tmp.values[2]-phi_tmp)
    return m1_tmp.values[0]

In [817]:
response_elec2 = np.abs((np.mean([get_amp3(i) for i in range(20)])*5/2))

phase response:  0.05958649574512087
phase response:  0.06522841719833039
phase response:  0.061150869564229104
phase response:  0.031148623613165505
phase response:  0.0457379790675958
phase response:  0.06292731538447338
phase response:  0.07447212969203054
phase response:  0.05141948588252454
phase response:  0.06559009603501931
phase response:  0.06413938062069624
phase response:  0.06827401346059858
phase response:  0.026751413730876195
phase response:  0.07097999359683005
phase response:  0.07088884778982418
phase response:  0.07577957674505242
phase response:  0.07232472269183621
phase response:  0.029821680029137543
phase response:  0.06754350911623419
phase response:  0.07621162415514782
phase response:  0.02772510447447729


In [1159]:
scalez = scale*(response_elec1+response_elec2)/(response_elec5+response_elec6)
scalez2 = scale*(response_elec1/2+response_elec2)/(response_elec5+response_elec6)
print('scales: ', scale, scalez, scalez2)

scales:  326.26 247.883865515 198.628453132


In [1171]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 41
def get_amp_z(i):
    
    fname = r'/data/old_trap/20210603/bead1/neutrality/w_height_fb_z_100V_sweep_m25v_50V_spin_xy_150V/turbombar_powfb_xyzcool_'+str(i)+'.h5'

    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 
                  'error_A': 1, 'error_f': 10, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [0, 10000], 'limit_f': [0, 100], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.other_data[2])[::decimate]
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
#     freq2 = np.fft.rfftfreq(len(bb.other_data[6]), d=1./5000)
#     fft_angles = np.angle(np.fft.rfft(bb.other_data[6]))

    phi_tmp = m1_tmp.values[2]
#     phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
#     print(phi_tmp, phi_tmp2, m1_tmp.values[0]*50, np.std(xx2)*np.sqrt(2)*50)
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp+0.05, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-100000, 100000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    xx2 = signal.filtfilt(b, a, bb.pos_data[2])[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [1172]:
get_amp_z(0)

-427.1107518944482

In [1184]:
amps_w_fb_spin_150V_xy_z_50V_100V_sweep_m50_50 = []
for i in range(300):
    print(i,'/500')
    try:
        amps_w_fb_spin_150V_xy_z_50V_100V_sweep_m50_50.append(get_amp_z(i))
    except:
        print("******")
amps_w_fb_spin_150V_xy_z_50V_100V_sweep_m50_50 = np.array(amps_w_fb_spin_150V_xy_z_50V_100V_sweep_m50_50)

0 /500
1 /500
2 /500
3 /500
4 /500
5 /500
6 /500
7 /500
8 /500
9 /500
10 /500
11 /500
12 /500
13 /500
14 /500
15 /500
16 /500
17 /500
18 /500
19 /500
20 /500
21 /500
22 /500
23 /500
24 /500
25 /500
26 /500
27 /500
28 /500
29 /500
30 /500
31 /500
32 /500
33 /500
34 /500
35 /500
36 /500
37 /500
38 /500
39 /500
40 /500
41 /500
42 /500
43 /500
44 /500
45 /500
46 /500
47 /500
48 /500
49 /500
50 /500
51 /500
52 /500
53 /500
54 /500
55 /500
56 /500
57 /500
58 /500
59 /500
60 /500
61 /500
62 /500
63 /500
64 /500
65 /500
66 /500
67 /500
68 /500
69 /500
70 /500
71 /500
72 /500
73 /500
74 /500
75 /500
76 /500
77 /500
78 /500
79 /500
80 /500
81 /500
82 /500
83 /500
84 /500
85 /500
86 /500
87 /500
88 /500
89 /500
90 /500
91 /500
92 /500
93 /500
94 /500
95 /500
96 /500
97 /500
98 /500
99 /500
100 /500
101 /500
102 /500
103 /500
104 /500
105 /500
106 /500
107 /500
108 /500
109 /500
110 /500
111 /500
112 /500
113 /500
114 /500
115 /500
116 /500
117 /500
Can't find/open HDF5 file : /data/old_trap/20210

Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45, in getdata
    f = h5py.File(fname,'r')
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 394, in __init__
    swmr=swmr)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 170, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 85, in h5py.h5f.open
OSError: Unable to open file (file signature not found)
Traceback (most recent call last):
  File "../../../../nadavp/opt_lev_analysis/lib/bead_data_funcs.py", line 45, in getdata
    f = h5py.File(fname,'r')
  File "/home/analysis_user/.local/lib/python3.6/site-packages/h5py/_hl/files.py", line 394, in __init__
    swmr=swmr)
  File "/home/analysis_user/.local/lib/python3.6/site-pac

120 /500
121 /500
122 /500
123 /500
124 /500
125 /500
126 /500
127 /500
128 /500
129 /500
130 /500
131 /500
132 /500
133 /500
134 /500
135 /500
136 /500
137 /500
138 /500
139 /500
140 /500
141 /500
142 /500
143 /500
144 /500
145 /500
146 /500
147 /500
148 /500
149 /500
150 /500
151 /500
152 /500
153 /500
154 /500
155 /500
156 /500
157 /500
158 /500
159 /500
160 /500
161 /500
162 /500
163 /500
164 /500
165 /500
166 /500
167 /500
168 /500
169 /500
170 /500
171 /500
172 /500
173 /500
174 /500
175 /500
176 /500
177 /500
178 /500
179 /500
180 /500
181 /500
182 /500
183 /500
184 /500
185 /500
186 /500
187 /500
188 /500
189 /500
190 /500
191 /500
192 /500
193 /500
194 /500
195 /500
196 /500
197 /500
198 /500
199 /500
200 /500
201 /500
202 /500
203 /500
204 /500
205 /500
206 /500
207 /500
208 /500
209 /500
210 /500
211 /500
212 /500
213 /500
214 /500
215 /500
216 /500
217 /500
218 /500
219 /500
220 /500
221 /500
222 /500
223 /500
224 /500
225 /500
226 /500
227 /500
228 /500
229 /500
230 /500
2

In [1187]:
data = amps_w_fb_spin_150V_xy_z_100V_sweep_m50_50
data2 = amps_w_fb_spin_150V_xy_z_50V_100V_sweep_m50_50
# data3 = amps_w_fb_spin_100V_xy_z_50V
_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(data)), data/scalez*voltage/voltage_z, label='z-100V, z-sweep -50V to 50V, xy-spin, w/ height fb')
ax.scatter(np.arange(len(data2)), data2/scalez2*voltage/voltage_z, label='z-50V-100V, z-sweep -50V to 50V, xy-spin, w/ height fb')
# ax.scatter(np.arange(len(data3)), data3/scalez*voltage/voltage_z, label='y-50V-bottom, xy-spin, w/ heigh fb')
ax.plot(np.arange(0,300), np.arange(0,300)*0, '--r')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()
print(np.mean(data/scalez*voltage/voltage_z), np.std(data/scalez*voltage/voltage_z)/np.sqrt(len(data)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

-0.0466597880376 0.00525377111518


In [840]:
data = amps_w_fb_spin_xy_z_50V_top_bottom
data2 = amps_w_fb_spin_xy_z_50V_top
data3 = amps_w_fb_spin_xy_z_50V_bottom
_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(data)), data/scalez*voltage/voltage_z, label='z-50V, xy-spin, w/ height fb')
ax.scatter(np.arange(len(data2)), data2/scalez*voltage/voltage_z*2, label='y-50V-top, xy-spin, w/ heigh fb')
ax.scatter(np.arange(len(data3)), data3/scalez*voltage/voltage_z*2, label='y-50V-bottom, xy-spin, w/ heigh fb')
ax.plot(np.arange(0,500), np.arange(0,500)*0, '--r')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()
print(np.mean(data/scalez*voltage/voltage_z), np.std(data/scalez*voltage/voltage_z)/np.sqrt(len(data)))

(12.368400161613192, 47.907286241551674, 247.883865515396, 326.26)